In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import warnings
import os
os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/Y4/Y4T1/PROJECT/ARDL-ECM/Code/Thai/quantstat')

# 1. Invalid frequency: ME
if not hasattr(pd.tseries.offsets, 'ME'):
    pd.tseries.offsets.ME = pd.tseries.offsets.MonthEnd

# 2. Resampler.sum axis error
from pandas.core.resample import Resampler
_original_resampler_sum = Resampler.sum
def _patched_resampler_sum(self, *args, **kwargs):
    if 'axis' in kwargs: kwargs.pop('axis')
    return _original_resampler_sum(self, *args, **kwargs)
Resampler.sum = _patched_resampler_sum

In [2]:
def create_rl_portfolio_report(file_name, stock_name, output_file='RL_LSTM_Portfolio_Report.html', shift=1):
    
    print(f"🚀 Processing: {file_name} -> {stock_name}")
    
    # 1. Load Data
    df = pd.read_excel(file_name)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # 2. Fetch Market Data
    start_date = df.index.min()
    end_date = df.index.max()
    price_data = yf.download(stock_name, start=start_date, end=end_date)
    
    if isinstance(price_data.columns, pd.MultiIndex):
        price_data.columns = price_data.columns.droplevel(1)

    # 3. Calculate Returns
    price_data['Returns'] = np.log(price_data['Close']).diff()
    
    # 4. Align Signal
    df['Aligned_Signal'] = df['Signal'].shift(shift)

    # Long-Only (ห้าม Short)
    df['Aligned_Signal'] = df['Aligned_Signal'].apply(lambda x: max(0, x))

    # 5. Merge Data
    portfolio = pd.merge(price_data['Returns'], df['Aligned_Signal'], left_index=True, right_index=True, how='inner')
    
    # Strategy Returns (ตอนนี้จะไม่มีการกำไรขาลงแล้ว)
    portfolio['Strategy_Returns'] = portfolio['Aligned_Signal'] * portfolio['Returns']
    portfolio.dropna(inplace=True)

    # 6. Generate Report
    print(f"📊 Generating Report: {output_file}")
    
    # Patch Resampler (ป้องกัน Error เดิม)
    if not getattr(Resampler.sum, 'is_patched', False):
        _original_resampler_sum = Resampler.sum
        def _patched_resampler_sum(self, *args, **kwargs):
            if 'axis' in kwargs: kwargs.pop('axis')
            return _original_resampler_sum(self, *args, **kwargs)
        _patched_resampler_sum.is_patched = True
        Resampler.sum = _patched_resampler_sum

    #####
    print(f'start date: {portfolio.index.min()}')
    print(f'end date: {portfolio.index.max()}')
    #####
    
    qs.extend_pandas()
    qs.reports.html(
        portfolio['Strategy_Returns'], 
        benchmark=portfolio['Returns'], 
        output=output_file,
        title=f'RL-LSTM Strategy vs {stock_name} (Long Only)',
        rf = 0.0115
    )
    
    print(f"✅ DONE! File saved: {output_file}") 

In [3]:
df = pd.read_excel('DL_Predictions_V33_Metrics_KKP.xlsx')
df['Aligned_Signal'] = df['Signal'].shift(1)
df['Aligned_Signal'] = df['Aligned_Signal'].apply(lambda x: max(0, x))
df = df[['Date', 'Aligned_Signal']]
df

,Date,Aligned_Signal
0,2024-01-03,0.0
1,2024-01-04,0.0
2,2024-01-05,0.0
3,2024-01-08,0.0
4,2024-01-09,0.0
...,...,...
480,2025-12-24,0.0
481,2025-12-25,0.0
482,2025-12-26,0.0
483,2025-12-29,0.0


In [4]:
df

,Date,Aligned_Signal
0,2024-01-03,0.0
1,2024-01-04,0.0
2,2024-01-05,0.0
3,2024-01-08,0.0
4,2024-01-09,0.0
...,...,...
480,2025-12-24,0.0
481,2025-12-25,0.0
482,2025-12-26,0.0
483,2025-12-29,0.0


In [6]:
file_name = 'DL_Predictions_V33_Metrics_KKP.xlsx'
output_file = 'KKP.html'
stock_name = 'KKP.BK'
create_rl_portfolio_report(file_name, stock_name, output_file)

/var/folders/rc/w7r40l35143bxwkzfxdygctc0000gn/T/ipykernel_53151/1613937577.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(stock_name, start=start_date, end=end_date)
/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
[*********************100%***********************]  1 of 1 completed

🚀 Processing: DL_Predictions_V33_Metrics_KKP.xlsx -> KKP.BK
📊 Generating Report: KKP.html
start date: 2024-01-04 00:00:00
end date: 2025-12-29 00:00:00


✅ DONE! File saved: KKP.html


/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/quantstats/_plotting/core.py:1016: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  port["Weekly"].ffill(inplace=True)
/Users/fulinq/.pyenv/versions/3.12.5/lib/python3.12/site-packages/quantstats/_plotting/core.py:1019: ChainedAssignmentError: A value is being set on a co